## Getting data from file 

In [8]:
import pandas as pd
gen_acc = pd.read_csv('users.csv')
gen_acc['bot'] = 0 
gen_acc.columns

Index(['id', 'name', 'screen_name', 'statuses_count', 'followers_count',
       'friends_count', 'favourites_count', 'listed_count', 'url', 'lang',
       'time_zone', 'location', 'default_profile', 'default_profile_image',
       'geo_enabled', 'profile_image_url', 'profile_banner_url',
       'profile_use_background_image', 'profile_background_image_url_https',
       'profile_text_color', 'profile_image_url_https',
       'profile_sidebar_border_color', 'profile_background_tile',
       'profile_sidebar_fill_color', 'profile_background_image_url',
       'profile_background_color', 'profile_link_color', 'utc_offset',
       'is_translator', 'follow_request_sent', 'protected', 'verified',
       'notifications', 'description', 'contributors_enabled', 'following',
       'created_at', 'timestamp', 'crawled_at', 'updated', 'test_set_1',
       'test_set_2', 'bot'],
      dtype='object')

In [1]:
import pandas as pd
gen_acc = pd.read_csv('users.csv')

gen_acc = gen_acc[['statuses_count', 'followers_count', 'friends_count', 'favourites_count', 
         'listed_count', 'default_profile', 'geo_enabled', 'profile_use_background_image', 
         'protected', 'verified']] 

gen_acc['bot'] = 0 

fake_acc = pd.read_csv('fakeusers.csv')

fake_acc = fake_acc[['statuses_count', 'followers_count', 'friends_count', 'favourites_count', 
         'listed_count', 'default_profile', 'geo_enabled', 'profile_use_background_image', 
         'protected', 'verified']] 

fake_acc['bot'] = 1


### Merging fake and genuine accounts

In [2]:
accounts = [gen_acc, fake_acc]

accounts = pd.concat(accounts)

accounts = accounts.fillna(0)


## Splitting data

In [3]:
input_x = accounts.iloc[:, 0:-1].values

input_y = accounts.iloc[:, -1].values



In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(input_x, input_y, test_size = 0.25, random_state = 0)


## Using Random forest classifier

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score


Using TensorFlow backend.


In [6]:
parameters = {'bootstrap': True,
              'min_samples_leaf': 3,
              'n_estimators': 50, 
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 6,
              'max_leaf_nodes': None}

In [7]:
RF_model = RandomForestClassifier(**parameters)
RF_model.fit(X_train, y_train)
RF_predictions = RF_model.predict(X_test)
score = accuracy_score(y_test ,RF_predictions)
print(score)

0.9894551845342706


In [8]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(RF_model, open(filename, 'wb'))

## Preprocessing Tweets

In [ ]:
import pandas as pd

gen_tweet2 = pd.read_csv('ds/tweets.csv')


In [ ]:
gen_tweet = twt

In [ ]:
gen_tweet.columns

In [ ]:
gen_tweet.head(5)

### Translate to lowercase

In [ ]:
# type(gen_tweet['text'])

import preprocessor as p


gen_tweet['text'] = gen_tweet['text'].apply(str)
gen_tweet['text'] = gen_tweet['text'].apply(p.clean)

In [ ]:
gen_tweet['text'] = gen_tweet['text'].apply(str)
gen_tweet['text'] = gen_tweet['text'].apply(lambda x: x.lower())
gen_tweet['text'] = gen_tweet['text'].apply(lambda x: x.strip())

#tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
# Remove URL
import re
gen_tweet['text'] = gen_tweet['text'].apply(lambda x:
                                          ' '.join(re.sub("(\w+:\/\/\S+)", " ", x).split()))

In [ ]:
gen_tweet['text']

In [ ]:
import numpy as np
labels = np.ones(248533)


In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
t = Tokenizer()
t.fit_on_texts(gen_tweet['text'])

In [ ]:
vocab_size = len(t.word_index) + 1

In [ ]:
encoded_docs = t.texts_to_sequences(gen_tweet['text'])

In [ ]:
from keras.preprocessing.sequence import pad_sequences

max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# print(padded_docs)

In [ ]:
# ds/glove.twitter.27B.25d.txt


embeddings_index = dict()
f = open('ds/glove.twitter.27B.100d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))

In [ ]:
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=2, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

# Tweet based classification

In [16]:
import pandas as pd

gen_tweet = pd.read_csv('tweet/gen.csv')

gen_tweet = gen_tweet[['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 
                     'text']]

gen_tweet['bot'] = 0 

f1 = pd.read_csv('tweet/f1.csv')
f2 = pd.read_csv('tweet/f2.csv')
f3 = pd.read_csv('tweet/f3.csv')


f1 = f1[['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 
                     'text']]

f2 = f2[['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 
                     'text']]

f3 = f3[['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 
                     'text']]


f1['bot'] = 1 
f2['bot'] = 1 
f3['bot'] = 1 

twt = [gen_tweet, f1, f2, f3]

twt = pd.concat(twt)

twt = twt.fillna(0)



# gen_tweet.shape

/home/krishna/.virtualenvs/cv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/krishna/.virtualenvs/cv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/krishna/.virtualenvs/cv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
